In [1]:
import pm4py
import os
import pandas as pd
import numpy as np
import copy
from collections import Counter
import random
import re
import warnings
warnings.filterwarnings("ignore")

In [10]:
def csv_to_xes(file, add_startend, id_column, act_column, start_time_column, end_time_column, resource_column):
    import_file = f'{file}.csv' #
    df = pd.read_csv(import_file)
#     print(df)
    id_column = id_column
    act_column = act_column
    start_time_column = start_time_column
    end_time_column = end_time_column
    resource_column = resource_column
    state_column = 'lifecycle:transition'
    time_column = 'time:timestamp'
    df[resource_column].fillna('no_resource', inplace=True)
    df.to_csv(f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/ConsultaDataMining201618_0.2/{file}.csv', index=False)
    if add_startend:
        # pre-processing
        mydata = pd.DataFrame()
        for key, group in df.groupby(id_column):
            group = add_start_end(df, group, act_column, resource_column, start_time_column, end_time_column)   # add start event in the beginning and end in the last 
            mydata = pd.concat([mydata, group], ignore_index=True)
        df = mydata

    
    df_start = df[[id_column, act_column, start_time_column, resource_column]].rename(columns={start_time_column:time_column})
    df_start[state_column] = 'start'
    df_start

    df_end = df[[id_column, act_column, end_time_column, resource_column]].rename(columns={end_time_column:time_column})
    df_end[state_column] = 'complete'

    df_start_end = pd.concat([df_start, df_end])
    df_start_end['index'] = df_start_end.index
    df_start_end[time_column] = pd.to_datetime(df_start_end[time_column])
    new = pd.DataFrame()
    for key, group in df_start_end.groupby(id_column):
        temp = group.sort_values(by=[time_column, 'index'], ascending=[True, True]).reset_index(drop=True)
        new = pd.concat([new,temp],ignore_index = True)
    del new['index']
    df = new
    
    
#     print(new.columns)
    df['concept:name'] = df[act_column].astype('str')
    df[id_column] = df[id_column].astype('str')
    print(df)
    df = df[[id_column, 'concept:name', state_column, resource_column, time_column, act_column]]
#     df = df[[id_column, act_column,  state_column, resource_column, time_column]]
    df = df.rename(columns = { resource_column: 'user', act_column: 'task'})
    
    pm4py.write.write_xes(df, f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/ConsultaDataMining201618_0.2/{file}.xes', case_id_key=id_column)

In [11]:
csv_to_xes('train_ConsultaDataMining201618', add_startend = False, id_column='caseid', act_column='task', start_time_column='start_timestamp', end_time_column='end_timestamp', resource_column='user')

       caseid                                   task      time:timestamp  \
0       81102                                  Start 2016-02-01 18:53:46   
1       81102                                  Start 2016-02-01 18:53:46   
2       81102  Traer informacion estudiante - banner 2016-02-01 18:53:46   
3       81102  Traer informacion estudiante - banner 2016-02-01 18:53:46   
4       81102         Radicar Solicitud Homologacion 2016-02-01 18:53:50   
...       ...                                    ...                 ...   
10971  129262                      Validar solicitud 2016-06-28 15:11:15   
10972  129262       Homologacion por grupo de cursos 2016-06-28 16:43:19   
10973  129262       Homologacion por grupo de cursos 2016-06-28 16:47:25   
10974  129262                                    End 2016-06-28 16:47:25   
10975  129262                                    End 2016-06-28 16:47:25   

        user lifecycle:transition                           concept:name  
0      Start

exporting log, completed traces ::   0%|          | 0/763 [00:00<?, ?it/s]

In [48]:
# info of train_data
if_csv = False #
import_file = f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/ConsultaDataMining201618_0.2/train_ConsultaDataMining201618.xes' #
id_column = 'caseid'
act_column = 'concept:name'
time_column = 'time:timestamp'
resource_column = 'user'
state_column = 'lifecycle:transition'

dirStr, ext = os.path.splitext(import_file)
file_name = dirStr.split("/")[-1]


data = pm4py.read.read_xes(import_file)

df = copy.deepcopy(pd.DataFrame(data,columns=[id_column, act_column, time_column, resource_column, state_column]))

parsing log, completed traces ::   0%|          | 0/763 [00:00<?, ?it/s]

In [13]:
df['waiting_time'] = None
df['process_time'] = None
df['last_complete_event'] = None
df['preceding_evts'] = None
df['paired_event'] = None
df['next'] = None
df['index'] = df.index
for key, group in df.groupby(id_column):
    flag = 0
    i = list(group.index)[0]
    preceding_evt = []
    not_complete_evt_idx = []
    j = 0
    df['waiting_time'].loc[i] = 0
    not_complete_evt_idx.append(i)
    i+=1
    while j<len(group)-1:
        j+=1
        cur_act = df.loc[i]
        if cur_act[state_column] == 'complete':
            flag = 0
            preceding_evt.append(cur_act.name)
            last_complete_evt_idx = i
            
            for each_idx in not_complete_evt_idx:
                to_pair = df.loc[each_idx]
                if (cur_act[act_column] == to_pair[act_column]) \
                and (cur_act[resource_column] == to_pair[resource_column]):
                    df['paired_event'].loc[i] = each_idx
                    not_complete_evt_idx.remove(each_idx)
                    df['process_time'].loc[i] = (df[time_column].loc[i] - df[time_column].loc[each_idx]).total_seconds()
                    df['last_complete_event'].loc[i] = df['last_complete_event'].loc[each_idx]
                    break
            
        else:
            if flag == 1:
                df['preceding_evts'].loc[i] = df['preceding_evts'].loc[i-1]
            else:
                df['preceding_evts'].loc[i] = preceding_evt#tuple(sorted, reverse=False))
            for each in df['preceding_evts'].loc[i]:
                if df['next'].loc[each] == None:
                    df['next'].loc[each] = [i]
                else:
                    temp = df['next'].loc[each]
                    temp.append(i)
                    df['next'].loc[each] = temp
            flag = 1
            preceding_evt = []
            not_complete_evt_idx.append(i)
            
            df['last_complete_event'].loc[i] = last_complete_evt_idx
            df['waiting_time'].loc[i] = (df[time_column].loc[i] - df[time_column].loc[last_complete_evt_idx]).total_seconds()
        i+=1

In [14]:
def fillna_parallel_data(bb):
    i = bb.index[0]
    j = 0 
    while j < len(bb):
        if bb[state_column].loc[i] == 'complete':
            paired = int(bb['paired_event'].loc[i])
            bb['process_time'].loc[paired] = bb['process_time'].loc[i] 
            bb['paired_event'].loc[paired] = i
            bb['next'].loc[paired] = bb['next'].loc[i]
            bb['waiting_time'].loc[i] = bb['waiting_time'].loc[paired] 
            bb['preceding_evts'].loc[i] = bb['preceding_evts'].loc[paired]
            bb['next'].loc[paired] = bb['next'].loc[i] 
        i+=1
        j+=1
fillna_parallel_data(df)


In [15]:
df = df[df[state_column]=='start']
df = df.rename(columns = {'concept:name':'activity'})
file = df[['activity', 'user', 'waiting_time', 'process_time']]
file

,activity,user,waiting_time,process_time
0,Start,Start,0,0
2,Traer informacion estudiante - banner,6426,0,0
4,Radicar Solicitud Homologacion,6426,4,966
6,Validar solicitud,22475,175060,119
8,Homologacion por grupo de cursos,1343,11384,2959
...,...,...,...,...
10966,Traer informacion estudiante - banner,19692,0,0
10968,Radicar Solicitud Homologacion,19692,1,0
10970,Validar solicitud,29277,137,43
10972,Homologacion por grupo de cursos,1007,5524,246


In [16]:
import math
file['waiting_time'] = file['waiting_time'].map(lambda x: math.log(x+1))
file['process_time'] = file['process_time'].map(lambda x: math.log(x+1))

In [17]:
file.reset_index(drop=True, inplace = True)
file

,activity,user,waiting_time,process_time
0,Start,Start,0.000000,0.000000
1,Traer informacion estudiante - banner,6426,0.000000,0.000000
2,Radicar Solicitud Homologacion,6426,1.609438,6.874198
3,Validar solicitud,22475,12.072890,4.787492
4,Homologacion por grupo de cursos,1343,9.340052,7.992945
...,...,...,...,...
5483,Traer informacion estudiante - banner,19692,0.000000,0.000000
5484,Radicar Solicitud Homologacion,19692,0.693147,0.000000
5485,Validar solicitud,29277,4.927254,3.784190
5486,Homologacion por grupo de cursos,1007,8.617039,5.509388


In [18]:
from collections import Counter
from sklearn import preprocessing
new_data = pd.DataFrame()
def get_info(df, new_df, column_name, column_type):
    if column_type == 'categorical':
        df[column_name].fillna(f'NO_{column_name}',inplace=True)
        dict_ = {}
        a = Counter(df[column_name])     
        dict_['name'] = column_name
        dict_['size'] = len(a.keys())
        dict_['type'] = column_type
        enc=preprocessing.LabelEncoder()
        enc=enc.fit(list(a.keys()))#训练LabelEncoder,将电脑，手表，手机编码为0,1,2
        data=enc.transform(df[column_name])#使用训练好的LabelEncoder对原数据进行编码，也叫归一化
        dict_['i2s'] = list(enc.classes_)
        new_df[column_name] = data
        return dict_
    else:
        dict_ = {}
        dict_['max'] = max(df[column_name])
        dict_['min'] = min(df[column_name])
        dict_['name'] = column_name
        dict_['type'] = column_type
        new_df[column_name] = df[column_name]
        return dict_


In [21]:
new_data = pd.DataFrame()
a = [ 'activity']
b = ['waiting_time', 'process_time']

json_input = {}
json_input['columns'] = [get_info(file, new_data, each, 'categorical') for each in a]
json_input['columns'].extend([get_info(file, new_data, each, 'continuous') for each in b])
json_input['problem_type'] = 'no'

In [22]:
new_data

,activity,waiting_time,process_time
0,10,0.000000,0.000000
1,11,0.000000,0.000000
2,7,1.609438,6.874198
3,15,12.072890,4.787492
4,5,9.340052,7.992945
...,...,...,...
5483,11,0.000000,0.000000
5484,7,0.693147,0.000000
5485,15,4.927254,3.784190
5486,5,8.617039,5.509388


In [32]:
for key, group in new_data.groupby('activity'):
    if key == 6.0:
        print(group)

      activity  waiting_time  process_time
9            6     11.980388           0.0
17           6     13.180521           0.0
53           6     12.064688           0.0
59           6     14.055274           0.0
89           6     13.516631           0.0
...        ...           ...           ...
5450         6      6.723832           0.0
5456         6     11.235035           0.0
5462         6      7.401231           0.0
5468         6     10.721393           0.0
5474         6     10.710499           0.0

[381 rows x 3 columns]


In [25]:
train_number = round(len(new_data)*0.8)
train_number

4390

In [27]:
import json
with open(f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/ConsultaDataMining201618_0.2/Consulta_act_time.json',"w") as f:
    json.dump(json_input,f)

# with open('test.json') as json_file:
#     a = json.load(json_file)

train = np.array(new_data.iloc[:train_number]).astype('float32')
train.shape

test = np.array(new_data.iloc[train_number:]).astype('float32')
test.shape


array_dict = {'train':train, 'test':test}
np.savez(f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/ConsultaDataMining201618_0.2/Consulta_act_time.npz', **array_dict)




In [28]:
gen = pd.read_csv('new_technique_train_ConsultaDataMining201618.csv')
gen_act = gen['act']
gen_act = gen_act.map(lambda x:json_input['columns'][0]['i2s'].index(x))
gen_act = np.array(gen_act).astype('float32')
gen_act

array_dict = {'gen':gen_act}
np.savez('Consulta_act_time_gen.npz', **array_dict)

In [59]:
gen_diffu_time = pd.read_csv(f'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/ConsultaDataMining201618_0.2/sample_1.csv')
gen_diffu_time = gen_diffu_time.rename(columns = {'0':'activity', '1':'waiting_time', '2':'process_time'})
gen_diffu_time['wait'] = gen_diffu_time['waiting_time'].map(lambda x: round(math.exp(x)-1))
gen_diffu_time['pro'] = gen_diffu_time['process_time'].map(lambda x: round(math.exp(x)-1))
gen_diffu_time

,activity,waiting_time,process_time,wait,pro
0,10.0,0.000000,0.001315,0,0
1,11.0,0.000000,0.000000,0,0
2,7.0,1.687904,0.129401,4,0
3,6.0,12.571671,0.000000,288274,0
4,3.0,0.000000,0.000968,0,0
...,...,...,...,...,...
1352,1.0,6.339101,9.744364,565,17057
1353,11.0,4.492982,0.002773,88,0
1354,7.0,2.171685,0.141510,8,0
1355,6.0,5.485216,0.000000,240,0


In [60]:
[int(each) for each in '(32,34)'.strip('(').strip(')').split(',')]

[32, 34]

In [61]:
gen = pd.read_csv(r'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/ConsultaDataMining201618_0.2/new_technique_train_ConsultaDataMining201618.csv')
gen['pro'] = gen_diffu_time['pro']
gen['wait'] = gen_diffu_time['wait']
gen = gen.set_index('Unnamed: 0')
gen['last'] = gen['last'].map(lambda x: [int(each) for each in x.strip('(').strip(')').split(',')] if x!='None' else x)
gen['next'] = gen['next'].map(lambda x: [int(each) for each in x.strip('(').strip(')').split(',')] if x!='None' else x)
# gen['last'] = gen['last'].map(lambda x: [x])
# gen['next'] = gen['next'].map(lambda x: [x])
gen

,caseid,act,index,last,next,if_para,start_timestamp,end_timestamp,resource,pro,wait
Unnamed: 0,,,,,,,,,,,
1,1,Start,1,None,[2],0,NaN,NaN,Start,0,0
2,1,Traer informacion estudiante - banner,2,[1],[3],0,NaN,NaN,Traer informacion estudiante - banner,0,0
3,1,Radicar Solicitud Homologacion,3,[2],[4],0,NaN,NaN,Radicar Solicitud Homologacion,0,4
4,1,Notificacion estudiante cancelacion soli,4,[3],[5],0,NaN,NaN,Notificacion estudiante cancelacion soli,0,288274
5,1,End,5,[4],None,0,NaN,NaN,End,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1733,191,Cancelar Solicitud,1733,[1732],[1734],0,NaN,NaN,Cancelar Solicitud,17057,565
1734,191,Traer informacion estudiante - banner,1734,[1733],[1735],0,NaN,NaN,Traer informacion estudiante - banner,0,88
1735,191,Radicar Solicitud Homologacion,1735,[1734],[1736],0,NaN,NaN,Radicar Solicitud Homologacion,0,8


In [62]:
# get the start time of each generated trace
ia_df = pd.read_csv(r'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/ConsultaDataMining201618_0.2/gen_ConsultaDataMining201618_1.csv')
ia_df
ia_list = []
for key, group in ia_df.groupby('caseid'):
    group['start_timestamp'] = group['start_timestamp'].map(lambda x: pd.to_datetime(x, format = '%Y-%m-%d  %H:%M:%S'))
    ia_list.append(min(group['start_timestamp']))
len(ia_list)

191

In [63]:
g = pd.DataFrame()
for key, group in gen.groupby('caseid'):
    group = group.set_index('index')
    for i in range(len(group)):
        cur = group.iloc[i]
        if i == 0:
            group['start_timestamp'].iloc[i] = ia_list[int(key)-1]
            group['end_timestamp'].iloc[i] = group['start_timestamp'].iloc[i] + pd.Timedelta(seconds=0)
        else:
            if len(cur['last']) > 1:
                last_time = max(pd.to_datetime(group['end_timestamp'].loc[list(cur['last'])]))
                group['start_timestamp'].iloc[i] = last_time + pd.Timedelta(seconds=group['wait'].iloc[i])
            else:
                last_time = pd.to_datetime(group['end_timestamp'].loc[cur['last'][0]])
                group['start_timestamp'].iloc[i] = last_time + pd.Timedelta(seconds=group['wait'].iloc[i])
            group['end_timestamp'].iloc[i] = group['start_timestamp'].iloc[i] + pd.Timedelta(seconds=group['pro'].iloc[i])
    g = pd.concat([g, group])
g

,caseid,act,last,next,if_para,start_timestamp,end_timestamp,resource,pro,wait
index,,,,,,,,,,
1,1,Start,None,[2],0,2016-02-01 15:01:55,2016-02-01 15:01:55,Start,0,0
2,1,Traer informacion estudiante - banner,[1],[3],0,2016-02-01 15:01:55,2016-02-01 15:01:55,Traer informacion estudiante - banner,0,0
3,1,Radicar Solicitud Homologacion,[2],[4],0,2016-02-01 15:01:59,2016-02-01 15:01:59,Radicar Solicitud Homologacion,0,4
4,1,Notificacion estudiante cancelacion soli,[3],[5],0,2016-02-04 23:06:33,2016-02-04 23:06:33,Notificacion estudiante cancelacion soli,0,288274
5,1,End,[4],None,0,2016-02-04 23:06:33,2016-02-04 23:06:33,End,0,0
...,...,...,...,...,...,...,...,...,...,...
1733,191,Cancelar Solicitud,[1732],[1734],0,2016-02-10 01:41:37,2016-02-10 06:25:54,Cancelar Solicitud,17057,565
1734,191,Traer informacion estudiante - banner,[1733],[1735],0,2016-02-10 06:27:22,2016-02-10 06:27:22,Traer informacion estudiante - banner,0,88
1735,191,Radicar Solicitud Homologacion,[1734],[1736],0,2016-02-10 06:27:30,2016-02-10 06:27:30,Radicar Solicitud Homologacion,0,8


In [64]:
g = g.rename(columns = {'act':'task'})
g['caseid'] = g['caseid'].map(lambda x: 'Case'+str(x))
g = g[['caseid', 'task','start_timestamp', 'end_timestamp', 'resource']]
g

,caseid,task,start_timestamp,end_timestamp,resource
index,,,,,
1,Case1,Start,2016-02-01 15:01:55,2016-02-01 15:01:55,Start
2,Case1,Traer informacion estudiante - banner,2016-02-01 15:01:55,2016-02-01 15:01:55,Traer informacion estudiante - banner
3,Case1,Radicar Solicitud Homologacion,2016-02-01 15:01:59,2016-02-01 15:01:59,Radicar Solicitud Homologacion
4,Case1,Notificacion estudiante cancelacion soli,2016-02-04 23:06:33,2016-02-04 23:06:33,Notificacion estudiante cancelacion soli
5,Case1,End,2016-02-04 23:06:33,2016-02-04 23:06:33,End
...,...,...,...,...,...
1733,Case191,Cancelar Solicitud,2016-02-10 01:41:37,2016-02-10 06:25:54,Cancelar Solicitud
1734,Case191,Traer informacion estudiante - banner,2016-02-10 06:27:22,2016-02-10 06:27:22,Traer informacion estudiante - banner
1735,Case191,Radicar Solicitud Homologacion,2016-02-10 06:27:30,2016-02-10 06:27:30,Radicar Solicitud Homologacion


In [65]:
g.to_csv(f'gen_diffu_time_{file_name}_2.csv', index=False)